In [0]:
from google.colab import auth

auth.authenticate_user()

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 16712 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1build3) ...
Setting up python-software-properties (0.96.24.17) ...
Setting up software-properties-common (0.96.24.17) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Processing triggers for dbus (1.10.22-1ubuntu1) ...
gpg: keybox '/tmp/tmp_28tqs8w/pubring.g

In [0]:
from oauth2client.client import GoogleCredentials

creds = GoogleCredentials.get_application_default()

In [0]:
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls

datalab  drive


In [0]:
import os

In [0]:
os.chdir("drive/ColabNotebooks/jigsaw-toxic-comment-classification-challenge/")

In [0]:
!ls

download_word_embendings.ipynb	submissions  try.ipynb	      work.ipynb
ft.wiki.en.pickle.zip		test_data    Untitled0.ipynb
sample_submission		train_data   wordEmbeddings


In [0]:
!pip install sklearn

  Running setup.py bdist_wheel for sklearn ... - done
  Stored in directory: /content/.cache/pip/wheels/d7/db/a3/1b8041ab0be63b5c96c503df8e757cf205c2848cf9ef55f85e
Successfully built sklearn


In [0]:
import time
from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd
np.random.seed(32)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer

Using TensorFlow backend.


In [0]:
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

In [0]:
import zipfile

In [0]:
zip_ref = zipfile.ZipFile("ft.wiki.en.pickle.zip" , 'r')
zip_ref.extractall("/content/")
zip_ref.close()

In [0]:
os.chdir("/content/")

In [0]:
!ls

datalab  drive	ft.wiki.en.pickle


In [0]:
os.chdir("drive/ColabNotebooks/jigsaw-toxic-comment-classification-challenge/")

In [0]:
import pickle

In [0]:
with open('/content/ft.wiki.en.pickle', 'rb') as handle:
    embedding_matrix = pickle.load(handle)

In [0]:
embedding_matrix.shape

(100000, 300)

In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [0]:
embed_size = 300
max_features = 100000
max_len = 200

In [0]:
!ls

download_word_embendings.ipynb	test_data   Untitled0.ipynb  X_valid.zip
ft.wiki.en.pickle.zip		test.zip    wordEmbeddings   Y_train.zip
sample_submission		train_data  work.ipynb	     Y_valid.zip
submissions			try.ipynb   X_train.zip


In [0]:
zip_ref = zipfile.ZipFile("X_train.zip" , 'r')
zip_ref.extractall("/content/")
zip_ref.close()

zip_ref = zipfile.ZipFile("X_valid.zip" , 'r')
zip_ref.extractall("/content/")
zip_ref.close()

zip_ref = zipfile.ZipFile("Y_train.zip" , 'r')
zip_ref.extractall("/content/")
zip_ref.close()

zip_ref = zipfile.ZipFile("Y_valid.zip" , 'r')
zip_ref.extractall("/content/")
zip_ref.close()

zip_ref = zipfile.ZipFile("test.zip" , 'r')
zip_ref.extractall("/content/")
zip_ref.close()

In [0]:
os.chdir("/content/")

In [0]:
!ls

datalab  ft.wiki.en.pickle  X_train.pickle  Y_train.pickle
drive	 test.pickle	    X_valid.pickle  Y_valid.pickle


In [0]:
with open('/content/X_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
    
with open('/content/Y_train.pickle', 'rb') as handle:
    Y_train = pickle.load(handle)

with open('/content/X_valid.pickle', 'rb') as handle:
    X_valid = pickle.load(handle)

with open('/content/Y_valid.pickle', 'rb') as handle:
    Y_valid = pickle.load(handle)
    
with open('/content/test.pickle', 'rb') as handle:
    test = pickle.load(handle)

In [0]:
X_train.shape

(143613, 200)

In [0]:
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1, save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

def build_model(lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    inp = Input(shape = (max_len,))
    x = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x = SpatialDropout1D(dr)(x)

    x = Bidirectional(GRU(units, return_sequences = True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])

    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, Y_train, batch_size = 128, epochs = 4, validation_data = (X_valid, Y_valid), verbose = 1, callbacks = [ra_val, check_point, early_stop])
    model = load_model(file_path)
    return model

In [0]:
!ls

datalab  ft.wiki.en.pickle  X_train.pickle  Y_train.pickle
drive	 test.pickle	    X_valid.pickle  Y_valid.pickle


In [0]:
t1 = time.time()

model = build_model(lr = 1e-3, lr_d = 0, units = 128, dr = 0.2)
pred = model.predict(test, batch_size = 1024, verbose = 1)

t2 = time.time()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 143613 samples, validate on 15958 samples
Epoch 1/4
 27264/143613 [====>.........................] - ETA: 7:11 - loss: 0.0894 - acc: 0.9731

143613/143613 [==============================] - 547s 4ms/step - loss: 0.0565 - acc: 0.9802 - val_loss: 0.0417 - val_acc: 0.9839

 ROC-AUC - epoch: 1 - score: 0.987605

Epoch 00001: val_loss improved from inf to 0.04170, saving model to best_model.hdf5
Epoch 2/4
  7552/143613 [>.............................] - ETA: 8:20 - loss: 0.0445 - acc: 0.9827

143613/143613 [==============================] - 551s 4ms/step - loss: 0.0435 - acc: 0.9833 - val_loss: 0.0425 - val_acc: 0.9829

 ROC-AUC - epoch: 2 - score: 0.990385

Epoch 00002: val_loss did not improve
Epoch 3/4
  3072/143613 [..............................] - ETA: 8:38 - loss: 0.0443 - acc: 0.9831

143613/143613 [==============================] - 550s 4ms/step - loss: 0.0409 - acc: 0.9839 - val_loss: 0.0390 - val_acc: 0.9843

 ROC-AUC - epoch: 3 - score: 0.990790

Epoch 00003: val_loss improved from 0.04170 to 0.03902, saving model to best_model.hdf5
Epoch 4/4
  1408/143613 [..............................] - ETA: 8:46 - loss: 0.0422 - acc: 0.9818

143613/143613 [==============================] - 552s 4ms/step - loss: 0.0386 - acc: 0.9848 - val_loss: 0.0397 - val_acc: 0.9840

 ROC-AUC - epoch: 4 - score: 0.991079

Epoch 00004: val_loss did not improve
 12288/153164 [=>............................] - ETA: 43s

153164/153164 [==============================] - 46s 299us/step


In [0]:
!ls

best_model.hdf5  ft.wiki.en.pickle  X_valid.pickle
datalab		 test.pickle	    Y_train.pickle
drive		 X_train.pickle     Y_valid.pickle


In [0]:
(t2 - t1)/60 

42.02823590834935

it took 42 minutes to train model with 4 epoche

In [0]:
os.chdir("drive/ColabNotebooks/jigsaw-toxic-comment-classification-challenge/")

In [0]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [0]:
submission = pd.read_csv("sample_submission/sample_submission.csv.zip")
submission[list_classes] = (pred)
submission.to_csv("submission.csv", index = False)

In [0]:
!ls

download_word_embendings.ipynb	submissions  try.ipynb	      work.ipynb
fast text model			test_data    Untitled0.ipynb
sample_submission		train_data   wordEmbeddings


In [0]:
!cp /content/best_model.hdf5 /content/drive/ColabNotebooks/jigsaw-toxic-comment-classification-challenge/

In [0]:
!ls

best_model.hdf5			submissions  Untitled0.ipynb
download_word_embendings.ipynb	test_data    wordEmbeddings
fast text model			train_data   work.ipynb
sample_submission		try.ipynb
